In [1]:
# Import the libraries
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegressionCV

from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Import data

In [2]:
#13k top spotify songs 
#df=pd.read_csv('data/df.csv', sep=",")
#users favorite tracks based on time 
df_fav=pd.read_csv('data/df_fav.csv', sep=",")
df=pd.read_csv('data/df_hot.csv', sep=",")

In [3]:
df.describe()

,length,artists,album,time_signature,danceability,acousticness,energy,tempo,instrumentalness,loudness,...,valence,speechiness,mode,Popularity,minutes,beat_density,Popularity_Rank,melody_complexity,instrumental_acoustic_ratio,favorite
count,1.259400e+04,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,...,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000
mean,2.385551e+05,4135.184215,4870.247816,4.502938,0.553316,0.315991,0.592348,119.010716,0.182240,-9.627065,...,0.469516,0.084973,0.672622,50.297933,3.975918,35.102960,6180.605606,465.286804,537.973461,0.009370
std,1.263497e+05,2385.110827,2834.519720,6.384068,0.189292,0.361614,0.283803,31.161578,0.322545,6.945035,...,0.428565,0.110423,0.684204,19.924747,2.105828,18.796769,3633.365412,132.168642,11978.910413,0.096346
min,9.750000e+03,0.000000,0.000000,0.000000,0.000000,0.000001,0.000568,0.000000,0.000000,-44.150000,...,0.000000,0.000000,0.000000,0.000000,0.162500,0.000000,1.000000,0.000000,0.000000,0.000000
25%,1.771040e+05,2094.250000,2406.000000,4.000000,0.428000,0.016200,0.413000,97.484000,0.000000,-10.962500,...,0.219000,0.036700,0.000000,39.000000,2.951733,24.422204,3045.500000,380.037000,0.000000,0.000000
50%,2.119465e+05,4201.500000,4908.500000,4.000000,0.571000,0.128000,0.655000,121.940000,0.000332,-7.214500,...,0.423000,0.047400,1.000000,52.000000,3.532442,33.464736,6092.000000,480.024000,0.005413,0.000000
75%,2.592970e+05,6213.750000,7320.750000,4.000000,0.693000,0.606750,0.827000,137.689500,0.180000,-5.198000,...,0.644000,0.079600,1.000000,65.000000,4.321617,42.873465,9370.500000,543.818000,0.922067,0.000000
max,4.170227e+06,8136.000000,9764.000000,96.000000,0.978000,0.996000,0.999000,219.446000,0.986000,1.177000,...,5.000000,1.000000,11.000000,100.000000,69.503783,574.662857,12343.000000,926.355000,747863.247863,1.000000


In [4]:
df.head(3)

,track_id,length,artists,album,time_signature,danceability,acousticness,energy,tempo,instrumentalness,...,valence,speechiness,mode,Popularity,minutes,beat_density,Popularity_Rank,melody_complexity,instrumental_acoustic_ratio,favorite
0,3rUGC1vUpkDG9CZFHMur1t,131872,7016,9646,1,0.75,0.2560,0.733,111.018,0.000000,...,0.844,0.0319,0,98.0,2.197867,50.511708,4.5,111.018,0.000000,0
1,7aqfrAY2p9BUSiupwk3svU,247444,2075,2891,4,0.47,0.0277,0.640,163.920,0.000000,...,0.248,0.3200,1,91.0,4.124067,39.747175,49.5,655.680,0.000000,0
2,7dJYggqjKo71KI9sLzqCs8,265360,2007,7025,4,0.75,0.2280,0.674,123.026,0.000089,...,0.392,0.0970,0,89.0,4.422667,27.817154,89.5,492.104,0.000391,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12594 entries, 0 to 12593
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   track_id                     12594 non-null  object 
 1   length                       12594 non-null  int64  
 2   artists                      12594 non-null  int64  
 3   album                        12594 non-null  int64  
 4   time_signature               12594 non-null  int64  
 5   danceability                 12594 non-null  float64
 6   acousticness                 12594 non-null  float64
 7   energy                       12594 non-null  float64
 8   tempo                        12594 non-null  float64
 9   instrumentalness             12594 non-null  float64
 10  loudness                     12594 non-null  float64
 11  liveness                     12594 non-null  float64
 12  key                          12594 non-null  float64
 13  valence         

In [6]:
df['favorite'].value_counts()

favorite
0    12476
1      118
Name: count, dtype: int64

In [7]:
print(df.shape)

(12594, 23)


In [8]:
#X = df.drop(columns=['favorite', 'track_id'])  # Drop 'favorite' and 'track_id' columns
X = df[['track_id','album', 'artists', 'length', 'danceability', 'tempo', 'energy', 'acousticness', 'time_signature','favorite']]
#'tempo','beat_density','speechiness','melody_complexity'
y = df['favorite']  # Select the 'favorite' column as the target variable

In [9]:
#X.to_csv('data/df_gbt.csv', index=False)

In [10]:
print(X.shape)
print(y.shape)

(12594, 10)
(12594,)


In [11]:
df = X
data = df 

In [12]:
data.head()

,track_id,album,artists,length,danceability,tempo,energy,acousticness,time_signature,favorite
0,3rUGC1vUpkDG9CZFHMur1t,9646,7016,131872,0.750,111.018,0.733,0.2560,1,0
1,7aqfrAY2p9BUSiupwk3svU,2891,2075,247444,0.470,163.920,0.640,0.0277,4,0
2,7dJYggqjKo71KI9sLzqCs8,7025,2007,265360,0.750,123.026,0.674,0.2280,4,0
3,4MjDJD8cW7iVeWInc2Bdyj,9691,742,267194,0.787,139.056,0.621,0.1500,4,0
4,5aIVCx5tnk0ntmdiinnYvw,9135,7579,200256,0.673,117.187,0.722,0.0856,4,0


#### cosine_similarity

In [13]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

# Normalizing the features
scaler = MinMaxScaler()
feature_columns = ['album', 'artists', 'length', 'danceability', 'tempo', 'energy', 'acousticness', 'time_signature','favorite']  # Replace with your actual feature columns
data[feature_columns] = scaler.fit_transform(data[feature_columns])

# Filter out the favorite songs
favorite_songs = data[data['favorite'] == 1]

# Compute the cosine similarity between all songs and the favorite songs
similarity_matrix = cosine_similarity(data[feature_columns], favorite_songs[feature_columns])

In [14]:
similarity_matrix

array([[0.45053113, 0.46878909, 0.43231257, ..., 0.56841684, 0.60499291,
        0.39834119],
       [0.36097053, 0.47209805, 0.42939416, ..., 0.52029287, 0.49569141,
        0.45501334],
       [0.51509373, 0.55422115, 0.50798512, ..., 0.63106832, 0.66415819,
        0.49025403],
       ...,
       [0.84222738, 0.94883738, 0.95350504, ..., 1.        , 0.96373831,
        0.89208886],
       [0.83469011, 0.9377988 , 0.9366106 , ..., 0.96373831, 1.        ,
        0.85618068],
       [0.81706845, 0.96992952, 0.95931494, ..., 0.89208886, 0.85618068,
        1.        ]])

In [15]:
#method 1

# Calculate the mean similarity score for each song to all favorite songs
mean_similarity_scores = np.mean(similarity_matrix, axis=1)

data['mean_similarity'] = mean_similarity_scores

In [16]:
from sklearn.cluster import KMeans

# Perform clustering based on similarity scores
kmeans = KMeans(n_clusters=10)  # number of clusters can be chosen based on your dataset
clusters = kmeans.fit_predict(similarity_matrix)

data['cluster_label'] = clusters

In [17]:
data

,track_id,album,artists,length,danceability,tempo,energy,acousticness,time_signature,favorite,mean_similarity,cluster_label
0,3rUGC1vUpkDG9CZFHMur1t,0.987915,0.862340,0.029353,0.766871,5.059012e-01,0.733582,0.257027,0.010417,0.0,0.512012,5
1,7aqfrAY2p9BUSiupwk3svU,0.296088,0.255039,0.057131,0.480573,7.469719e-01,0.640436,0.027810,0.041667,0.0,0.461976,7
2,7dJYggqjKo71KI9sLzqCs8,0.719480,0.246681,0.061438,0.766871,5.606208e-01,0.674490,0.228915,0.041667,0.0,0.581753,1
3,4MjDJD8cW7iVeWInc2Bdyj,0.992524,0.091200,0.061878,0.804703,6.336684e-01,0.621406,0.150601,0.041667,0.0,0.569698,1
4,5aIVCx5tnk0ntmdiinnYvw,0.935580,0.931539,0.045789,0.688139,5.340129e-01,0.722565,0.085943,0.041667,0.0,0.490068,5
...,...,...,...,...,...,...,...,...,...,...,...,...
12589,5R2rsbwCDXORX2tLfprRmM,0.591356,0.001844,0.037638,0.737219,0.000000e+00,0.508229,0.318272,0.750000,1.0,0.960850,2
12590,6DZz58CbF0AKw5PCKqNKcM,0.238120,0.001475,0.072262,0.449898,5.741731e-09,0.465161,0.691767,0.000000,1.0,0.780821,2
12591,3CA9pLiwRIGtUBiMjbZmRw,0.730746,0.000737,0.048311,0.598160,4.420222e-07,0.909859,0.089457,0.854167,1.0,0.941917,2
12592,27GmP9AWRs744SzKcpJsTZ,0.943466,0.000737,0.047141,0.871166,0.000000e+00,0.553300,0.056123,0.770833,1.0,0.940393,2


In [56]:
#mean_similarity df
#data_1 = data.drop(['track_id', 'cluster_label'], axis=1)
#cluster_label df
#data_2 = data.drop(['track_id', 'mean_similarity'], axis=1)

#'cluster_label'
#"mean_similarity"
X = data[['tempo', 'danceability', 'time_signature', 'length']]
track_ids = data['track_id']



# 'album', 'artists', 'length', 'danceability', 'tempo', 'energy', 'acousticness', 'time_signature' 

y = data['favorite']  #  target variable


In [41]:
X

,tempo,danceability,time_signature,length
0,0.51,0.77,0.01,0.03
1,0.75,0.48,0.04,0.06
2,0.56,0.77,0.04,0.06
3,0.63,0.80,0.04,0.06
4,0.53,0.69,0.04,0.05
...,...,...,...,...
12589,0.00,0.74,0.75,0.04
12590,0.00,0.45,0.00,0.07
12591,0.00,0.60,0.85,0.05
12592,0.00,0.87,0.77,0.05


### Train & Testing data set 

In [59]:
from imblearn.over_sampling import SMOTE

# Shuffle your dataset 
shuffle_df = df.sample(frac=1)

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test, track_ids_train, track_ids_test = train_test_split(
    X, y, track_ids, test_size=0.2, random_state=42)

#data imbalance
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X, y) 

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(y_train.shape)
print(y_test.shape)

(24952,)
(2519,)


##### fix the imbalanced data set 

In [43]:
y_train.value_counts() 

favorite
0.00    12476
1.00    12476
Name: count, dtype: int64

In [44]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(24952, 4)
(24952,)
(2519, 4)
(2519,)


#### Classification 

In [45]:
import lazypredict
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,predictions=True, ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [01:25<00:00,  2.94s/it]

[LightGBM] [Info] Number of positive: 12476, number of negative: 12476
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000798 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 24952, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

In [46]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'Gradient Boosting': GradientBoostingClassifier()
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f'{name} - Accuracy: {accuracy_score(y_test, y_pred)}, Precision: {precision_score(y_test, y_pred)}, Recall: {recall_score(y_test, y_pred)}, F1 Score: {f1_score(y_test, y_pred)}')


Logistic Regression - Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F1 Score: 1.0
Random Forest - Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F1 Score: 1.0
SVM - Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F1 Score: 1.0
Gradient Boosting - Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F1 Score: 1.0


In [60]:
# Assuming RandomForest performs the best
best_model = RandomForestClassifier()
best_model.fit(X_train, y_train)
threshold =  0.5

# Predict probabilities
probabilities = best_model.predict_proba(X_test)[:, 1]  # Probability of being a favorite

# Recommend songs with the highest probabilities
recommendations = X_test[probabilities > threshold]  # Set a threshold


In [61]:
# Choose and train your model (e.g., Random Forest)
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict probabilities for the test set
probabilities = model.predict_proba(X_test)[:, 1]


In [62]:
# Create a DataFrame with track IDs and probabilities
recommendations_df = pd.DataFrame({
    'track_id': track_ids_test,
    'probability': probabilities
})

# Set a threshold for recommendations
threshold = 0.5  # adjust as needed

# Filter for recommended songs
recommended_songs = recommendations_df[recommendations_df['probability'] > threshold]

# Output the track IDs of recommended songs
recommended_track_ids = recommended_songs['track_id'].tolist()
print(recommended_track_ids)


['0nAZGkBGKQCXyaoSJfRhC1', '27GmP9AWRs744SzKcpJsTZ', '43CIWpJaJHRcI4mUSjqgc2', '6dkffLWo31KnXmwCLsgA2K', '3JWh19WsndEFg0UFRFeTiU', '1kBgQ2cBNONDo7ABrvmRfu', '17IBDpAl3J39swaoPlfGpp', '1L0OmJV0LnPDWuHCIGmBV6', '40EXFgNMOIQRQOwJZ8mJkC', '2YSzYUF3jWqb9YP9VXmpjE', '6rjuKpPydT2SxN15TZpV7r', '4aMxs74PXoQg2SLgjypcvb', '3LtpKP5abr2qqjunvjlX5i', '3oZZfVvLJuDYc4Vn3A63Fw', '3aQem4jVGdhtg116TmJnHz', '1cByP16xx3Bikp74r1TW5f', '6jy9yJfgCsMHdu2Oz4BGKX', '05nbZ1xxVNwUTcGwLbp7CN', '4rXLjWdF2ZZpXCVTfWcshS', '0xl1w2q4VLojeXp4JfazPL']


In [63]:
recommended_track_ids

['0nAZGkBGKQCXyaoSJfRhC1',
 '27GmP9AWRs744SzKcpJsTZ',
 '43CIWpJaJHRcI4mUSjqgc2',
 '6dkffLWo31KnXmwCLsgA2K',
 '3JWh19WsndEFg0UFRFeTiU',
 '1kBgQ2cBNONDo7ABrvmRfu',
 '17IBDpAl3J39swaoPlfGpp',
 '1L0OmJV0LnPDWuHCIGmBV6',
 '40EXFgNMOIQRQOwJZ8mJkC',
 '2YSzYUF3jWqb9YP9VXmpjE',
 '6rjuKpPydT2SxN15TZpV7r',
 '4aMxs74PXoQg2SLgjypcvb',
 '3LtpKP5abr2qqjunvjlX5i',
 '3oZZfVvLJuDYc4Vn3A63Fw',
 '3aQem4jVGdhtg116TmJnHz',
 '1cByP16xx3Bikp74r1TW5f',
 '6jy9yJfgCsMHdu2Oz4BGKX',
 '05nbZ1xxVNwUTcGwLbp7CN',
 '4rXLjWdF2ZZpXCVTfWcshS',
 '0xl1w2q4VLojeXp4JfazPL']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
# Other imports for different ML models...

# Load the cosine similarity matrix
cosine_sim_loaded = np.load('path_to_saved_matrix/cosine_similarity_matrix.npy')

# Example: Using cosine similarity scores as features
# Let's say each item is represented by its similarity scores to all other items
X = cosine_sim_loaded
y = your_data['favorite']  # Replace with your target variable

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a model (example: Random Forest)
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the performance metrics
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")


#### track_id to song name 

In [65]:
# Declare the credentials
client_id = '2a7adde1b07a4bf9a5fd750d6ff6f28d'
client_secret = '74a5c5859a0548ccae791419cc29c8fa'
redirect_uri='http://localhost:7777/callback'
username = 'ethan.falcao'
# Authorization flow
scope = 'user-top-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [66]:
import requests
def get_track_info(track_id, token):
    url = f'https://api.spotify.com/v1/tracks/{track_id}'
    headers = {
        'Authorization': f'Bearer {token}',
    }
    response = requests.get(url, headers=headers)
    return response.json()

In [67]:
track_ids = recommended_track_ids
for track_id in track_ids:
    track_info = get_track_info(track_id, token)
    print(track_info['name'])


A Keeper
Jumpman
Bad Boy (with Young Thug)
Gist
Minute
Sidenote
Another Late Night (feat. Lil Yachty)
On The Radar Freestyle
Who Told You (feat. Drake)
IDGAF (feat. Yeat)
500lbs
Turban
Doja
Polar Opposites
What’s Next
Hëavyweight
Papi’s Home
Myself
fukumean
Sacrifices


### Train & Testing data set 

In [27]:
# Shuffle your dataset 
shuffle_df = df.sample(frac=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(y_train.shape)
print(y_test.shape)

(10075,)
(2519,)


In [28]:
X_train

array([[ 0.12604664, -1.4274222 , -0.93266627, ..., -1.37343819,
        -1.18360404, -0.04340723],
       [-0.0131337 , -1.47689497,  0.54698844, ..., -0.802509  ,
         0.2664497 , -0.043406  ],
       [-0.47055525,  1.62102266, -0.17055922, ...,  1.13514828,
         0.71913583, -0.043359  ],
       ...,
       [-0.53727231,  0.00812658,  1.00142351, ...,  0.250146  ,
         1.02310471, -0.04340711],
       [ 0.8796991 , -1.30667511, -1.08544133, ..., -1.29361012,
        -0.11879787, -0.04340723],
       [-0.82818581,  0.18170053,  1.29176685, ..., -1.24976673,
         0.11564508, -0.04340718]])

In [29]:
#make sure the columns are assigned correctly 
#X_train = pd.DataFrame(X_train, columns=['album', 'artists', 'length', 'danceability', 'tempo', 'energy', 'acousticness', 'time_signature'])
#X_test = pd.DataFrame(X_test, columns=['album', 'artists', 'length', 'danceability', 'tempo', 'energy', 'acousticness', 'time_signature'])
#

In [30]:
#X_train.head(3)

##### fix the imbalanced data set 

In [31]:
#from imblearn.over_sampling import SMOTE
# Train / Split Data
#oversample = SMOTE()

#X_train, y_train = oversample.fit_resample(X, y) 

In [32]:
from imblearn.over_sampling import BorderlineSMOTE

bsmote = BorderlineSMOTE(random_state = 42) 
X_train, y_train = bsmote.fit_resample(X, y) 

In [33]:
y_train.value_counts()

favorite
0    12476
1    12476
Name: count, dtype: int64

In [34]:
y_test.value_counts()

favorite
0    2499
1      20
Name: count, dtype: int64

In [35]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(24952, 21)
(24952,)
(2519, 21)
(2519,)


## ML Algo

In [36]:
import lazypredict
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,predictions=True, ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

100%|██████████| 29/29 [02:26<00:00,  5.07s/it]

[LightGBM] [Info] Number of positive: 12476, number of negative: 12476
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001068 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4877
[LightGBM] [Info] Number of data points in the train set: 24952, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

### KNN's  

In [36]:
# Assuming '_pca' is already fitted on your data

# Get the feature names
feature_names = X.columns  # Assuming 'df' is your DataFrame

# Get the indices of the top features in each principal component
top_feature_indices = [np.abs(_pca.components_[i]).argsort()[::-1][:8] for i in range(components)]

# Get the names of the top features in each principal component
selected_features = [[feature_names[j] for j in top_feature_indices[i]] for i in range(components)]

# Print the selected features for each component
for i, features in enumerate(selected_features):
    print(f"Top features for component {i}: {features}")


NameError: name 'components' is not defined

In [ ]:
# Initialize the KNN model and fit it to the training data
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors
knn.fit(X_train, y_train)

# Predict the target variable for the test set
y_pred = knn.predict(X_test)


In [ ]:
# Evaluate the model (you can use classification metrics to assess its performance)
# For instance, you can use accuracy_score, classification_report, confusion_matrix, etc.
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Further evaluation
print(classification_report(y_test, y_pred, zero_division=1)) 
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9920603414053196
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2499
           1       1.00      0.00      0.00        20

    accuracy                           0.99      2519
   macro avg       1.00      0.50      0.50      2519
weighted avg       0.99      0.99      0.99      2519

[[2499    0]
 [  20    0]]


In [ ]:
from sklearn.model_selection import cross_val_score, KFold

# Assuming X and y are your features and target variable

# Initialize the KNN model
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors

# Define the number of folds for cross-validation
num_folds = 5

# Initialize KFold
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(knn, X, y, cv=kf)

# Output the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())

Cross-Validation Scores: [0.99166336 0.99285431 0.99086939 0.98769353 0.99245433]
Mean Accuracy: 0.9911069832734286


### Descision tree 

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score, classification_report


# Create and fit the Decision Tree classifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
# Make predictions
predictions = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9920603414053196


In [ ]:
report = classification_report(y_test, predictions)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      2499
           1       0.00      0.00      0.00        20

    accuracy                           0.99      2519
   macro avg       0.50      0.50      0.50      2519
weighted avg       0.98      0.99      0.99      2519



### Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(pt)

def recommend(book_name):
    index = np.where(pt.index == book_name)[0][0]
    similar_books = sorted(enumerate(similarity_scores[index]),key= lambda x: x[1], reverse =True)[1:6]
    
    for i in similar_books:
        print(pt.index[i[0]])

## PipeLine

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

components = 8  # Define the number of components for PCA

_sc = StandardScaler()
_pca = PCA(n_components=components)
#_model = KNeighborsClassifier(n_neighbors=5)
_model =DecisionTreeClassifier()

Pipe = Pipeline([
    ('std_scaler', _sc),
    ('pca', _pca),
    ('regressor', _model)
])

In [ ]:
# perform a split
random_state = 30

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, 
                     test_size=0.2,
                     shuffle=True, 
                     random_state=random_state)
    
# train the model using the PCA components
Pipe.fit(X_train,y_train)


Pipeline(steps=[('std_scaler', StandardScaler()), ('pca', PCA(n_components=8)),
                ('regressor', DecisionTreeClassifier())])

In [ ]:
Pipe.score(X_test,y_test)

1.0

### Predict Songs 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12594 entries, 0 to 12593
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   track_id                     12594 non-null  object 
 1   length                       12594 non-null  int64  
 2   artists                      12594 non-null  int64  
 3   album                        12594 non-null  int64  
 4   time_signature               12594 non-null  int64  
 5   danceability                 12594 non-null  float64
 6   acousticness                 12594 non-null  float64
 7   energy                       12594 non-null  float64
 8   tempo                        12594 non-null  float64
 9   instrumentalness             12594 non-null  float64
 10  loudness                     12594 non-null  float64
 11  liveness                     12594 non-null  float64
 12  key                          12594 non-null  float64
 13  valence         

In [ ]:
# Predicting if a song is a favorite
prob_preds = Pipe.predict_proba(df.drop(['favorite','track_id'], axis=1))
threshold = 0.30 # define threshold here
preds = [1 if prob_preds[i][1]> threshold else 0 for i in range(len(prob_preds))]
df['prediction'] = preds

In [ ]:
df['prediction']

0        0
1        0
2        0
3        0
4        0
        ..
12589    1
12590    1
12591    1
12592    1
12593    1
Name: prediction, Length: 12594, dtype: int64

In [ ]:
df['prediction'].value_counts()

NameError: name 'df' is not defined

### Building the playlist from recommended songs upload to Spotify 

In [ ]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2

In [ ]:
# Declare the credentials
client_id = '2a7adde1b07a4bf9a5fd750d6ff6f28d'
client_secret = '74a5c5859a0548ccae791419cc29c8fa'
redirect_uri='http://localhost:7777/callback'
username = 'ethan.falcao'

In [ ]:
# Once the Authorisation is complete, we just need to `sp` to call the APIs
scope = 'user-top-read playlist-modify-private playlist-modify-public'
token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [ ]:
# Creating a function that builds a playlist in the user's spotify account
def create_playlist(sp, username, playlist_name, playlist_description):
    playlists = sp.user_playlist_create(username, playlist_name, description = playlist_description)

In [ ]:
create_playlist(sp, username, 'ML PLaylist', 'This playlist was created using machine learning!')

In [ ]:
# Function to check if the playlist was created successfully
def fetch_playlists(sp, username):
    """
    Returns the user's playlists.
    """
        
    id = []
    name = []
    num_tracks = []
    
    # Make the API request
    playlists = sp.user_playlists(username)
    for playlist in playlists['items']:
        id.append(playlist['id'])
        name.append(playlist['name'])
        num_tracks.append(playlist['tracks']['total'])

    # Create the final df   
    df_playlists = pd.DataFrame({"id":id, "name": name, "#tracks": num_tracks})
    return df_playlists

In [ ]:
fetch_playlists(sp,username).head()

,id,name,#tracks
0,4pAo35sQk7E0v0PumqL6L5,ML PLaylist,0
1,1steg8eX8EvQoRt3O251il,ML PLaylist,118
2,7q9MublCR8vYmsEfZI3QkS,Your New Jams,118
3,1AJFPBjF116BRPJsbjoOn7,July 2023,20
4,1oxbP11MEsbkUqyMlaIYac,October 2023,27


In [ ]:
# Getting the playlist ID of the most recently made playlist so we can add songs to it later
playlist_id = fetch_playlists(sp,username)['id'][0]

In [ ]:
# Function to add selected songs to playlist
def enrich_playlist(sp, username, playlist_id, playlist_tracks):
    index = 0
    results = []
    
    while index < len(playlist_tracks):
        results += sp.user_playlist_add_tracks(username, playlist_id, tracks = playlist_tracks[index:index + 50])
        index += 50

In [ ]:
# Adding songs to playlist
list_track = df.loc[df['prediction']  == 1]['track_id']
enrich_playlist(sp, username, playlist_id, list_track)